In [1]:
import psycopg2
import pandas as pd
from urllib.parse import quote_plus

# Connection details
host = "dvdrental.postgres.database.azure.com"
database = "postgres"
user = "gbadmin"
password = "Catherin3!"
port = 5432

# Connect to the database
try:
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port,
        sslmode='require',
        connect_timeout=10
    )
    cursor = conn.cursor()
    print("✅ Connected to Azure PostgreSQL successfully!")
    
    # Get current column names
    cursor.execute("""
        SELECT column_name, data_type 
        FROM information_schema.columns 
        WHERE table_name = 'jobs' 
        ORDER BY ordinal_position;
    """)
    columns = cursor.fetchall()
    print(f"\n📋 Current columns in 'jobs' table: {len(columns)}")
    for col_name, col_type in columns:
        print(f"  - {col_name} ({col_type})")
    
except Exception as e:
    print(f"❌ Connection error: {e}")
    conn = None
    cursor = None

✅ Connected to Azure PostgreSQL successfully!

📋 Current columns in 'jobs' table: 23
  - employerId (bigint)
  - employerName (text)
  - jobId (bigint)
  - jobTitle (text)
  - locationName (text)
  - minimumSalary (double precision)
  - maximumSalary (double precision)
  - yearlyMinimumSalary (double precision)
  - yearlyMaximumSalary (double precision)
  - currency (text)
  - salaryType (text)
  - salary (text)
  - datePosted (text)
  - expirationDate (text)
  - externalUrl (text)
  - jobUrl (text)
  - partTime (boolean)
  - fullTime (boolean)
  - contractType (text)
  - jobDescription (text)
  - applicationCount (bigint)
  - skills (text)
  - skills_list (text)


In [2]:
# Rename all camelCase columns to lowercase
if conn:
    try:
        # List of columns to rename (based on current table structure)
        column_renames = [
            ('employerId', 'employerid'),
            ('employerName', 'employername'),
            ('jobId', 'jobid'),
            ('jobTitle', 'jobtitle'),
            ('locationName', 'locationname'),
            ('minimumSalary', 'minimumsalary'),
            ('maximumSalary', 'maximumsalary'),
            ('yearlyMinimumSalary', 'yearlyminimumsalary'),
            ('yearlyMaximumSalary', 'yearlymaximumsalary'),
            ('salaryType', 'salarytype'),
            ('datePosted', 'dateposted'),
            ('expirationDate', 'expirationdate'),
            ('externalUrl', 'externalurl'),
            ('jobUrl', 'joburl'),
            ('partTime', 'parttime'),
            ('fullTime', 'fulltime'),
            ('contractType', 'contracttype'),
            ('jobDescription', 'jobdescription'),
            ('applicationCount', 'applicationcount'),
            ('skills_list', 'skills_list')  # already lowercase
        ]
        
        # Rename each column
        renamed_count = 0
        skipped_count = 0
        for old_name, new_name in column_renames:
            if old_name == new_name:
                skipped_count += 1
                continue
            try:
                cursor.execute(f'ALTER TABLE jobs RENAME COLUMN "{old_name}" TO {new_name};')
                print(f"✅ Renamed: {old_name} → {new_name}")
                renamed_count += 1
            except psycopg2.errors.UndefinedColumn as e:
                print(f"⚠️  Column '{old_name}' doesn't exist, skipping...")
                conn.rollback()
            except Exception as e:
                print(f"❌ Error renaming {old_name}: {e}")
                conn.rollback()
        
        # Commit all successful changes
        conn.commit()
        print(f"\n✅ Successfully renamed {renamed_count} columns! ({skipped_count} already lowercase)")
        
    except Exception as e:
        print(f"❌ Error during column renaming: {e}")
        conn.rollback()
else:
    print("⚠️  No database connection available")

✅ Renamed: employerId → employerid
✅ Renamed: employerName → employername
✅ Renamed: jobId → jobid
✅ Renamed: jobTitle → jobtitle
✅ Renamed: locationName → locationname
✅ Renamed: minimumSalary → minimumsalary
✅ Renamed: maximumSalary → maximumsalary
✅ Renamed: yearlyMinimumSalary → yearlyminimumsalary
✅ Renamed: yearlyMaximumSalary → yearlymaximumsalary
✅ Renamed: salaryType → salarytype
✅ Renamed: datePosted → dateposted
✅ Renamed: expirationDate → expirationdate
✅ Renamed: externalUrl → externalurl
✅ Renamed: jobUrl → joburl
✅ Renamed: partTime → parttime
✅ Renamed: fullTime → fulltime
✅ Renamed: yearlyMaximumSalary → yearlymaximumsalary
✅ Renamed: salaryType → salarytype
✅ Renamed: datePosted → dateposted
✅ Renamed: expirationDate → expirationdate
✅ Renamed: externalUrl → externalurl
✅ Renamed: jobUrl → joburl
✅ Renamed: partTime → parttime
✅ Renamed: fullTime → fulltime
✅ Renamed: contractType → contracttype
✅ Renamed: jobDescription → jobdescription
✅ Renamed: applicationCount → 

In [3]:
# Verify the changes and load table into DataFrame
if conn:
    try:
        # Get updated column names
        cursor.execute("""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_name = 'jobs' 
            ORDER BY ordinal_position;
        """)
        columns = cursor.fetchall()
        print(f"📋 Updated columns in 'jobs' table:")
        for col_name, col_type in columns:
            print(f"  - {col_name} ({col_type})")
        
        # Load the entire table into a DataFrame
        df_jobs = pd.read_sql("SELECT * FROM jobs;", conn)
        print(f"\n✅ Loaded jobs table: {len(df_jobs)} rows, {len(df_jobs.columns)} columns")
        
    except Exception as e:
        print(f"❌ Error loading table: {e}")
        df_jobs = None
else:
    print("⚠️  No database connection available")
    df_jobs = None

# Display the DataFrame
df_jobs

📋 Updated columns in 'jobs' table:
  - employerid (bigint)
  - employername (text)
  - jobid (bigint)
  - jobtitle (text)
  - locationname (text)
  - minimumsalary (double precision)
  - maximumsalary (double precision)
  - yearlyminimumsalary (double precision)
  - yearlymaximumsalary (double precision)
  - currency (text)
  - salarytype (text)
  - salary (text)
  - dateposted (text)
  - expirationdate (text)
  - externalurl (text)
  - joburl (text)
  - parttime (boolean)
  - fulltime (boolean)
  - contracttype (text)
  - jobdescription (text)
  - applicationcount (bigint)
  - skills (text)
  - skills_list (text)

✅ Loaded jobs table: 100 rows, 23 columns


/var/folders/dr/y9rwzxgd23xcg66p8znfy6sh0000gn/T/ipykernel_31156/1337212716.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_jobs = pd.read_sql("SELECT * FROM jobs;", conn)


,employerid,employername,jobid,jobtitle,locationname,minimumsalary,maximumsalary,yearlyminimumsalary,yearlymaximumsalary,currency,...,expirationdate,externalurl,joburl,parttime,fulltime,contracttype,jobdescription,applicationcount,skills,skills_list
0,645147,Vermelo RPO,56061055,Application Support Analyst 6 month FTC,Bretton,26500.00,28500.00,26500.0,28500.0,GBP,...,24/12/2025,https://markerstudygroup.talosats-careers.com/...,https://www.reed.co.uk/jobs/application-suppor...,False,True,Permanent,<p><strong>Application Support Analyst – 6 mo...,12,{Python},Python
1,524744,Hays Specialist Recruitment Limited,56060755,Credit Risk Analyst,Coventry,45000.00,50000.00,45000.0,50000.0,GBP,...,24/12/2025,None,https://www.reed.co.uk/jobs/credit-risk-analys...,False,True,Permanent,<p><strong>Your new company</strong><br />Sal...,42,"{SQL,""Power BI"",Python}","SQL, Power BI, Python"
2,524744,Hays Specialist Recruitment Limited,56060755,Credit Risk Analyst,Coventry,45000.00,50000.00,45000.0,50000.0,GBP,...,24/12/2025,None,https://www.reed.co.uk/jobs/credit-risk-analys...,False,True,Permanent,<p><strong>Your new company</strong><br />Sal...,42,"{SQL,""Power BI"",Python}","SQL, Power BI, Python"
3,612529,Randstad Technologies,56058423,Senior Business Analyst - Records Management,London,600.00,642.00,156000.0,166920.0,GBP,...,23/12/2025,None,https://www.reed.co.uk/jobs/senior-business-an...,False,True,Contract,<p><strong>Senior Business Analyst - Records ...,141,"{SQL,Jira}","SQL, Jira"
4,601455,Pertemps Cardiff,56055119,Business Analyst,Cardiff,23.38,23.38,45591.0,45591.0,GBP,...,23/12/2025,None,https://www.reed.co.uk/jobs/business-analyst/5...,False,True,Temporary,<strong>Business Analyst – Data &amp; Process ...,75,{Azure},Azure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,332753,Noir,56095276,Technical Business Analyst - London / Remote,London,60000.00,90000.00,60000.0,90000.0,GBP,...,31/12/2025,None,https://www.reed.co.uk/jobs/technical-business...,False,True,Permanent,<p><strong>Technical Business Analyst - Londo...,140,"{SQL,API,Jira}","SQL, API, Jira"
96,17696,TXP Technology x People,56096596,SC Cleared Service Performance Analyst,London,340.00,390.00,88400.0,101400.0,GBP,...,31/12/2025,None,https://www.reed.co.uk/jobs/sc-cleared-service...,False,True,Contract,<p><strong>SC Cleared Service Analyst</strong...,33,{},
97,524744,Hays Specialist Recruitment Limited,56001743,Business Process Analyst SAP S/4 Hana,Cheshire,500.00,500.00,130000.0,130000.0,GBP,...,11/12/2025,None,https://www.reed.co.uk/jobs/business-process-a...,False,True,Permanent,<p><strong>Business Process Analyst </strong>...,42,{Excel},Excel
98,563845,Adecco,55966652,Collections Analyst £500/d REMOTE Customer Cre...,London,500.00,500.00,130000.0,130000.0,GBP,...,03/12/2025,None,https://www.reed.co.uk/jobs/collections-analys...,False,True,Contract,<p><strong>Customer Collections Analyst | COL...,60,"{SQL,""Power BI"",Python}","SQL, Power BI, Python"


In [ ]:
# Close the database connection when done
if conn:
    cursor.close()
    conn.close()
    print("✅ Database connection closed")